In [1]:
from glob import glob
import pandas as pd
from pismragis.processing import convert_netcdf_to_dataframe, ncfile2dataframe
from pismragis.analysis import prepare_df

In [2]:
infiles = glob("../tests/data/ts_gris_g1200m_v2023_RAGIS_id_*_1980-1-1_2020-1-1.nc")
df = convert_netcdf_to_dataframe(infiles, add_vars=False)
df.to_parquet("../tests/data/test_scalar.parquet")
df.to_csv("../tests/data/test_scalar.csv")



Converting netcdf files to pandas.DataFrame
-------------------------------------------


Processing files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 17.82it/s]


Conversion finished in 1 seconds
-------------------------------------------


In [3]:
df = convert_netcdf_to_dataframe(infiles, add_vars=False, resample="yearly")
df.to_parquet("../tests/data/test_scalar_YM.parquet")
df.to_csv("../tests/data/test_scalar_YM.csv")



Converting netcdf files to pandas.DataFrame
-------------------------------------------


Processing files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 133.61it/s]

Conversion finished in 0 seconds
-------------------------------------------


In [4]:
df = convert_netcdf_to_dataframe(infiles, add_vars=True)
df.to_parquet("../tests/data/test_scalar_add_vars.parquet")



Converting netcdf files to pandas.DataFrame
-------------------------------------------


Processing files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 108.99it/s]


Conversion finished in 0 seconds
-------------------------------------------


In [5]:
infile = "../tests/data/ts_gris_g1200m_v2023_RAGIS_id_0_1980-1-1_2020-1-1.nc"

In [6]:
df = ncfile2dataframe(infile, add_vars=False)
df.to_parquet("../tests/data/test_scalar_file.parquet")
df.to_csv("../tests/data/test_scalar_file.csv")

In [7]:
df = ncfile2dataframe(infile, add_vars=False, resample="yearly")
df.to_parquet("../tests/data/test_scalar_file_YM.parquet")
df.to_csv("../tests/data/test_scalar_file_YM.csv")

In [8]:
df = ncfile2dataframe(infile, add_vars=True)
df.to_parquet("../tests/data/test_scalar_file_add_vars.parquet")
df.to_csv("../tests/data/test_scalar_file_add_vars.csv")